In [61]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
from sklearn.model_selection import train_test_split, cross_val_score

# Videogames Analysis

## Datasets

### Steam Games

The Steam dataset originates from https://data.world/craigkelly/steam-game-data and was created by Craig Kelly. The dataset includes ownership data on over 40,000 videogames on the Steam digital distribution platform, and features such as genre, full text descriptions (written by the publisher/developer of the game), descriptive tags, supported languages, release dates, and reviewer scores. I'm interested in this dataset because I intend to publish a game within the next few years and I'm curious as to whether or not certain features are correlated with game popularity.


In [3]:
steam = pd.read_csv("data/games-features.csv")

In [4]:
print(steam.columns)

Index(['QueryID', 'ResponseID', 'QueryName', 'ResponseName', 'ReleaseDate',
       'RequiredAge', 'DemoCount', 'DeveloperCount', 'DLCCount', 'Metacritic',
       'MovieCount', 'PackageCount', 'RecommendationCount', 'PublisherCount',
       'ScreenshotCount', 'SteamSpyOwners', 'SteamSpyOwnersVariance',
       'SteamSpyPlayersEstimate', 'SteamSpyPlayersVariance',
       'AchievementCount', 'AchievementHighlightedCount', 'ControllerSupport',
       'IsFree', 'FreeVerAvail', 'PurchaseAvail', 'SubscriptionAvail',
       'PlatformWindows', 'PlatformLinux', 'PlatformMac', 'PCReqsHaveMin',
       'PCReqsHaveRec', 'LinuxReqsHaveMin', 'LinuxReqsHaveRec',
       'MacReqsHaveMin', 'MacReqsHaveRec', 'CategorySinglePlayer',
       'CategoryMultiplayer', 'CategoryCoop', 'CategoryMMO',
       'CategoryInAppPurchase', 'CategoryIncludeSrcSDK',
       'CategoryIncludeLevelEditor', 'CategoryVRSupport', 'GenreIsNonGame',
       'GenreIsIndie', 'GenreIsAction', 'GenreIsAdventure', 'GenreIsCasual',
       'G

In [5]:
steam.head()

,QueryID,ResponseID,QueryName,ResponseName,ReleaseDate,RequiredAge,DemoCount,DeveloperCount,DLCCount,Metacritic,...,LegalNotice,Reviews,SupportedLanguages,Website,PCMinReqsText,PCRecReqsText,LinuxMinReqsText,LinuxRecReqsText,MacMinReqsText,MacRecReqsText
0,10,10,Counter-Strike,Counter-Strike,Nov 1 2000,0,0,1,0,88,...,,,English French German Italian Spanish Simplifi...,None,Minimum: 500 mhz processor 96mb ram 16mb video...,,Minimum: Linux Ubuntu 12.04 Dual-core from Int...,,Minimum: OS X Snow Leopard 10.6.3 1GB RAM 4GB...,
1,20,20,Team Fortress Classic,Team Fortress Classic,Apr 1 1999,0,0,1,0,0,...,,,English French German Italian Spanish,None,Minimum: 500 mhz processor 96mb ram 16mb video...,,Minimum: Linux Ubuntu 12.04 Dual-core from Int...,,Minimum: OS X Snow Leopard 10.6.3 1GB RAM 4GB...,
2,30,30,Day of Defeat,Day of Defeat,May 1 2003,0,0,1,0,79,...,,,English French German Italian Spanish,http://www.dayofdefeat.com/,Minimum: 500 mhz processor 96mb ram 16mb video...,,Minimum: Linux Ubuntu 12.04 Dual-core from Int...,,Minimum: OS X Snow Leopard 10.6.3 1GB RAM 4GB...,
3,40,40,Deathmatch Classic,Deathmatch Classic,Jun 1 2001,0,0,1,0,0,...,,,English French German Italian Spanish,None,Minimum: 500 mhz processor 96mb ram 16mb video...,,Minimum: Linux Ubuntu 12.04 Dual-core from Int...,,Minimum: OS X Snow Leopard 10.6.3 1GB RAM 4GB...,
4,50,50,Half-Life: Opposing Force,Half-Life: Opposing Force,Nov 1 1999,0,0,1,0,0,...,,,English French German Korean,None,Minimum: 500 mhz processor 96mb ram 16mb video...,,Minimum: Linux Ubuntu 12.04 Dual-core from Int...,,Minimum: OS X Snow Leopard 10.6.3 1GB RAM 4GB...,


In [6]:
print(steam.iloc[100])

QueryID                                                          3170
ResponseID                                                       3170
QueryName                             King's Bounty: Armored Princess
ResponseName                           Kings Bounty: Armored Princess
ReleaseDate                                               Nov 19 2009
                                          ...                        
PCRecReqsText       Recommended:OS: Windows XP/Vista Processor: 3 ...
LinuxMinReqsText                                                     
LinuxRecReqsText                                                     
MacMinReqsText      OS: Mac OSX 10.6 or higher                Proc...
MacRecReqsText                                                       
Name: 100, Length: 78, dtype: object


In [7]:
steam['PriceCurrency'].value_counts()

USD    10739
        2618
Name: PriceCurrency, dtype: int64

In [8]:
steam['ResponseName'].value_counts().head(30)

Train Simulator 2016                              9
Jewel Quest Pack                                  7
Jagged Alliance 2 Gold                            5
SiN Episodes: Emergence                           5
F.E.A.R.                                          5
Mahjong Quest Collection                          5
Cake Mania Collection                             5
Tom Clancys Rainbow Six(r) Siege                  5
Crazy Machines 1.5                                5
BRINK                                             5
RIP - Trilogy(tm)                                 5
Westward Collection                               5
Super Granny Collection                           5
Tradewinds Classics                               5
Crysis Warhead(r)                                 3
Shank                                             3
Portal 2                                          3
The Sims(tm) 3                                    3
Civilization IV: Beyond the Sword                 3
Trine 2: Com

In [9]:
#steam['ExtUserAcctNotice'].value_counts()
#steam['DRMNotice'].value_counts()
#steam['Background'].value_counts()
#steam['HeaderImage'].value_counts()
#steam['Reviews'].value_counts()
#steam['PCMinReqsText'].value_counts()
steam['ReleaseDate'].value_counts()

Aug 22 2016      241
Aug 26 2016      152
                  87
Sep 9 2016        77
Sep 7 2016        68
                ... 
Mar 19 2012        1
Oct 1 2012         1
Sep 25 2008        1
Early October      1
Mar 27 2007        1
Name: ReleaseDate, Length: 2117, dtype: int64

## Notes on Steam dataset

Category is already encoded.

It looks like all prices are in USD, though some are unlabeled I'm just going to assume that they are USD and the webscraper didn't retrieve them correctly.

For now textual descriptions such as Background, AboutText, SupportedLanguages, and MinReqsText will all be removed as I do not yet know how to perform a bag-of-words or word-vector style feature decomposition on these. I will probably add them back in later when I've done more research on ohw to do this.

Links like Background or HeaderImage will also be removed.

Note that these removed columns probably do actually contain very valuable data to making an accurate prediction, it's just that I currently do not know how to incorporate them into regression and/or it would be infeasible to donwload the content of all of these links for 40,000 examples in a short amount of time. I might come back to that though??

There appear to be duplicate games??? I'll have to think of a creative way to merge these.

In [35]:
def steam_preprocess(df):
    ret = df.drop(labels=[
        'QueryID',
        'ResponseID',
        'QueryName',
        'ResponseName', # Could be useful for 'catchiness' or 'length', but that's a whole can of worms
        'SupportEmail',
        'SupportURL',
        'Background', # boolean
        'ShortDescrip',
        'DetailedDescrip',
        'AboutText',
        'PriceCurrency',
        'DRMNotice', # useful to convert to boolean value
        'ExtUserAcctNotice', # useful to convert to boolean value
        'HeaderImage', # boolean
        'LegalNotice', # boolean
        'Reviews', # boolean
        'SupportedLanguages', # Could separate up into a series of encoded variables, remove for now
        'Website', # boolean
        'PCMinReqsText', # All of these could have sentiment analysis done on them
        'PCRecReqsText',
        'LinuxMinReqsText',
        'LinuxRecReqsText',
        'MacMinReqsText',
        'MacRecReqsText',
        'SteamSpyPlayersEstimate', # The next three are dead giveaways and would give the model too much information.
        'SteamSpyPlayersVariance',
        'SteamSpyOwnersVariance', # Maybe could be incorporated into evaluation in some way
    ], axis=1)
    
    ret['ReleaseDate'] = pd.to_datetime(ret['ReleaseDate'], errors='coerce')
    
    # I would rather remove games that have no release date rather than impute them. Knowing steam, they probably aren't released yet, and imputation makes no sense here.
    # I might replace it with iterative imputation, but as it stands I'm losing like 400 games out of the 40,000 samples. I'd rather just not train on them.
    ret.dropna(subset=['ReleaseDate'], inplace=True)
    
    ret['ReleaseYear'] = pd.DatetimeIndex(ret['ReleaseDate']).year
    ret['ReleaseMonth'] = pd.DatetimeIndex(ret['ReleaseDate']).month
    ret['ReleaseDay'] = pd.DatetimeIndex(ret['ReleaseDate']).day
    ret = ret.drop(['ReleaseDate'], axis=1)
    return ret

steam_cleaned = steam_preprocess(steam)
#steam_cleaned.dtypes # MINDBLOWINGLY, SHOCKINGLY CLEAN

In [66]:
steam_cleaned.shape

(12872, 53)

In [11]:
X = steam_cleaned.drop(['SteamSpyOwners'], axis=1)
y = steam_cleaned['SteamSpyOwners']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

In [69]:
from sklearn.linear_model import SGDRegressor, LinearRegression

# No hyperparameter tuning

start = time.time()
reg = SGDRegressor(alpha=0, max_iter=10000).fit(X_train, y_train)
print("Time elapsed: ", time.time() - start)

reg.coef_

Time elapsed:  0.1640174388885498


array([ 1.39098810e+12,  2.93885866e+10,  8.43524008e+10,  5.45791656e+12,
        1.32013793e+12,  2.19745800e+11,  5.99540643e+10, -5.99758202e+11,
        4.89763859e+10, -1.52912601e+11,  2.02310136e+12,  1.72715351e+12,
        1.04488489e+11,  9.96744925e+10, -1.84808846e+09,  4.07300659e+10,
       -1.17322555e+10,  8.54628500e+08, -1.58096244e+10,  4.59416817e+09,
        5.96478292e+10,  1.22801297e+11,  8.01905673e+10,  1.19316514e+11,
        5.05680618e+10,  2.86896612e+10,  2.30924111e+10,  2.10305014e+10,
        6.20797487e+10,  1.91830786e+10,  3.69097927e+08, -1.35612148e+10,
        2.07905006e+10,  3.54039558e+09,  3.79946007e+10, -9.88167816e+09,
        1.74466802e+11,  6.56921233e+10, -1.60553329e+11, -6.74190875e+09,
        5.44405799e+10,  5.10184839e+10, -5.69369121e+10,  9.87110282e+10,
       -1.53964110e+09,  5.98451787e+10,  4.28137740e+10,  1.15398286e+12,
       -3.23726590e+11,  4.40867407e+11, -1.48969702e+11, -2.22542199e+10])

In [64]:
reg = LinearRegression().fit(X_train, y_train)

reg.coef_

array([ 4.20256403e+03, -7.63442301e+04,  2.90665190e+04, -2.27728811e+02,
        2.50194944e+03,  4.26126994e+04, -1.70327480e+04,  4.38456307e+01,
        8.18796316e+04, -9.38328409e+02,  5.65521655e+02, -1.06361745e+03,
       -3.09671805e+04,  1.59895424e+05, -3.19749632e+05,  1.77358819e+04,
       -1.92346838e+03, -6.58036770e+04,  1.62201595e+05,  6.08913778e+04,
       -8.21429279e+04,  4.32664565e+04, -5.57853252e+04, -7.83650053e+04,
        1.68617751e+04, -4.06928884e+04, -2.21971576e+05, -4.81333461e+04,
        1.57414436e+05,  7.10343929e+04,  4.04852633e+05,  2.48698854e+06,
        4.37185789e+04,  7.36768985e+04, -1.43228875e+05, -5.24268691e+04,
        1.00636289e+05,  1.82773742e+03,  6.37832475e+03, -5.42835224e+02,
        5.04873491e+03,  3.04002883e+04,  2.47132874e+04,  3.76975868e+05,
       -6.42186124e+03, -3.44353182e+04, -2.68425127e+05,  2.32245804e+02,
       -1.41604594e+03, -7.20753926e+04, -2.64897022e+03, -2.35702064e+03])

In [56]:
from sklearn.utils.estimator_checks import check_estimator
from sklearn.base import BaseEstimator, RegressorMixin
# Custom linear regression

class GhettoLinearRegression(BaseEstimator, RegressorMixin):
    coef_ = None
    
    def _pad_column(self, X): # Adds column of 1's at the beginning for the intercept
        X = X.to_numpy(dtype='float')
        return np.insert(X, 0, 1, axis=1)
        #return np.append(X, 1, axis=1)
    
    def fit(self, X, y):
        X = self._pad_column(X)
        self.coef_ = np.linalg.inv(X.T.dot(X)).dot(X.T.dot(y))
        return self
    
    def predict(self, X):
        X = self._pad_column(X)
        return X.dot(self._coef)
    
    def score(self, X, y):
        pred = self.predict(X)
        return (pred - y)**2

In [63]:
start = time.time()
reg = GhettoLinearRegression().fit(X_train, y_train)
print("Time elapsed: ", time.time() - start)

reg.coef_

Time elapsed:  0.08312153816223145


array([ 1.45427091e+08,  4.20256403e+03, -7.63442301e+04,  2.90665190e+04,
       -2.27728811e+02,  2.50194944e+03,  4.26126994e+04, -1.70327480e+04,
        4.38456307e+01,  8.18796316e+04, -9.38328409e+02,  5.65521655e+02,
       -1.06361745e+03, -3.09671805e+04,  1.59895424e+05, -3.19749632e+05,
        1.77358819e+04, -1.92346839e+03, -6.58036767e+04,  1.62201595e+05,
        6.08913778e+04, -8.21429279e+04,  4.32664565e+04, -5.57853252e+04,
       -7.83650053e+04,  1.68617750e+04, -4.06928884e+04, -2.21971576e+05,
       -4.81333461e+04,  1.57414436e+05,  7.10343929e+04,  4.04852633e+05,
        2.48698854e+06,  4.37185789e+04,  7.36768985e+04, -1.43228875e+05,
       -5.24268691e+04,  1.00636289e+05,  1.82773742e+03,  6.37832475e+03,
       -5.42835225e+02,  5.04873491e+03,  3.04002883e+04,  2.47132874e+04,
        3.76975868e+05, -6.42186124e+03, -3.44353182e+04, -2.68425127e+05,
        2.32245804e+02, -1.41604594e+03, -7.20753926e+04, -2.64897022e+03,
       -2.35702064e+03])